In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.shape

(404290, 6)

In [4]:
df.head()


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [9]:
# taking random 50K questions from the dataset
new_df = df.sample(50000)

In [10]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [11]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
298484,Why are so many people obsessed with the flat ...,Why are there still people who think that the ...
289691,Which is the best earphone under 1000rs? Is so...,Which headphones/earphones are the best under ...
2309,How do you get a job at an embassy or consulate?,How can you get a job in any embassy?
295891,Can we deposit a self cheque in the bank?,What is the clearance time for cheques deposit...
11297,"How do I learn about computer programming, net...",How can I learn about computers?


In [12]:
#using count vectorizer to extract features from the data
from sklearn.feature_extraction.text import CountVectorizer

questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=5000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [13]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape


(50000, 10000)

In [14]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
298484,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
289691,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2309,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
295891,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11297,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241623,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
347228,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
305909,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


In [15]:
# adding the is_duplicate column from the new_df dataset to temp_df dataset
temp_df['is_duplicate'] = new_df['is_duplicate']

In [16]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,4991,4992,4993,4994,4995,4996,4997,4998,4999,is_duplicate
298484,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
289691,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2309,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
295891,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11297,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [17]:
#splitting the data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(temp_df.iloc[:,0:-1].values, temp_df.iloc[:,-1].values, test_size=0.3, random_state = 3)


In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)

0.7494

In [20]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.7321333333333333